Elliptic Curve Factoring Algorithm:

We want to implement a factoring algorithm for a number n for some elliptic curve E mod n using the ECF Algorithm.

The general plan goes:

- Pick n
- Randomly pick $P=(x_0, y_0), a$ randomly $\in \mathbb{Z}_n$, solve for $b \simeq y_0^2 - x_0^2 - ax_0$ (mod n)
- Repeatedly add P, performing the extended euclidean algorithm to find the slope of $K_iP$ 
- Wait for $GCD(A,n) \neq 1$
- Output that factor 

In [8]:
import pandas as pd
import random 
import numpy as np
import plotly.express as px
from random import randint
from math import gcd
import gmpy2
from gmpy2 import mpz, is_prime, gcdext, invert
import sympy
from sympy import isprime, factorint, mod_inverse
from dataclasses import dataclass
from typing import Optional

In [9]:
def random_curve_and_point(n):
    while True:
        x = random.randint(1, n - 1)
        y = random.randint(1, n - 1)
        a = random.randint(1, n - 1)

        # Solve for b so (x, y) lies on the curve
        rhs = (pow(y, 2, n) - pow(x, 3, n) - a * x) % n
        b = rhs

        # Create curve object
        curve = Curve(A=a, B=b, N=n)

        # Optional: check discriminant
        delta = curve.discriminant()
        d = gcd(delta, n)
        if d != 1 and d != n:
            # Found a nontrivial factor
            return None, None, d

        point = Point(x=x, y=y, curve=curve)
        return curve, point, None

@dataclass
class Curve:
    A: int
    B: int
    N: int  # modulus (composite number to factor)

    def discriminant(self):
        return (4 * self.A**3 + 27 * self.B**2) % self.N

@dataclass
class Point:
    x: Optional[int]
    y: Optional[int]
    curve: Curve

    def is_infinity(self):
        return self.x is None and self.y is None

def point_add(P: Point, Q: Point):
    curve = P.curve
    n = curve.N

    if P.is_infinity():
        return Q, None
    if Q.is_infinity():
        return P, None

    x1, y1 = P.x, P.y
    x2, y2 = Q.x, Q.y

    if x1 == x2 and (y1 + y2) % n == 0:
        return Point(None, None, curve), None

    if x1 == x2 and y1 == y2:
        numerator = (3 * x1 * x1 + curve.A) % n
        denominator = (2 * y1) % n
    else:
        numerator = (y2 - y1) % n
        denominator = (x2 - x1) % n

    d = gcd(denominator, n)
    if 1 < d < n:
        return None, d

    try:
        inv_denominator = gmpy2.invert(denominator, n)
    except ZeroDivisionError:
        return None, gcd(denominator, n)

    lam = (numerator * inv_denominator) % n
    x3 = (lam * lam - x1 - x2) % n
    y3 = (lam * (x1 - x3) - y1) % n

    return Point(x3, y3, curve), None

def ecm_walk(n: int, max_iter: int = 5000):
    # Step 1: Generate curve and point
    curve, point, factor = random_curve_and_point(n)

    if factor:
        print(f"Nontrivial factor found during curve generation: {factor}")
        return factor

    print(f"Curve A = {curve.A}, B = {curve.B}")
    print(f"P = ({point.x}, {point.y}) on curve mod {curve.N}")

    # Step 2: Start doubling and walking
    next_p, factor = point_add(point, point)
    if factor:
        print(f"Found factor during first addition: {factor}")
        return factor

    print(f"2P = ({next_p.x}, {next_p.y})")

    i = 2
    while i <= max_iter:
        i += 1
        next_p, factor = point_add(point, next_p)

        if factor:
            print(f"Found factor of n: {factor}")
            return factor
        elif next_p:
            print(f'{i}P = ({next_p.x}, {next_p.y})')
        else:
            print("Point addition failed unexpectedly.")
            return None

    raise ValueError(f"Exceeded max iterations ({max_iter}) without finding a factor.")

In [10]:
ecm_walk(randint(1,30000),50000)

Nontrivial factor found during curve generation: 15


15